In [1]:
import os
import math
import time
import timeit
import numpy as np
import sys
import tarfile
import datetime
import multiprocessing
import scriptCreator
from pathlib import Path
import shutil
import fcntl



dicosPath = "/ceph/work/NTHU-qubit/LYT/tSDRG_random"
scopionPath = "/home/aronton/tSDRG_random"

if os.path.isdir(dicosPath):
    tSDRG_path = dicosPath
    group_path = dicosPath
    
if os.path.isdir(scopionPath):
    tSDRG_path = scopionPath
    group_path = scopionPath
    
sourcelist = {"ZL":"ZL.csv", "energy":"energy.csv", "seed":"s_re_seed.csv",\
    "corr1":"_".join(["L_re","P_re","m_re","s_re","corr1.csv"]), "corr2":"_".join(["L_re","P_re","m_re","s_re","corr2.csv"]),\
    "ZLI":"ZLI.csv", "ZLC":"ZLC.csv", "w_loc":"w_loc.csv", "J_list":"J_list.csv", "dimerization":"dimerization.csv",\
    "string":"_".join(["L_re","P_re","m_re","s_re","string.csv"])
    }

grouplist = {"ZL":"_".join(["L_re","P_re","m_re","s_re","ZL.txt"]), "energy":"_".join(["L_re","P_re","m_re","s_re","energy.txt"]),\
    "corr1":"_".join(["L_re","P_re","m_re","s_re","corr1.txt"]), "corr2":"_".join(["L_re","P_re","m_re","s_re","corr2.txt"]),\
    "ZLI":"_".join(["L_re","P_re","m_re","s_re","ZLI.txt"]), "ZLC":"_".join(["L_re","P_re","m_re","s_re","ZLC.txt"]),\
    "w_loc":"_".join(["L_re","P_re","m_re","s_re","w_loc.txt"]), "J_list":"_".join(["L_re","P_re","m_re","s_re","J_list.txt"]),\
    "string":"_".join(["L_re","P_re","m_re","s_re","string.txt"]), "seed":"_".join(["L_re","P_re","m_re","s_re","seed.txt"]),\
    "dimerization":"_".join(["L_re","P_re","m_re","s_re","dimerization.txt"])
    }

tarlist = {
    "ZL":"_".join(["ZL","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "energy":"_".join(["energy","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "corr1":"_".join(["corr1","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "corr2":"_".join(["corr2","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "ZLI":"_".join(["ZLI","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "ZLC":"_".join(["ZLC","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "w_loc":"_".join(["w_loc","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "J_list":"_".join(["J_list","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "string":"_".join(["string","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "seed":"_".join(["seed","L_re","P_re","m_re.txt"]),
    "dimerization":"_".join(["dimerization","L_re","P_re","m_re.txt"])
    }
            

def checkInside(s, f, sample, phys):
    with open(f,"r") as a:
        a = a.readlines()    
        if a[0].strip() == phys:
            del a[0]
        data = [(v.split(":")[0].strip(),(v.split(":")[1].replace("\n"," ").strip())) for i,v in enumerate(a)]
        sorted_data = sorted(data, key=lambda x: int(x[0]))
    if s == sorted_data[sample+1][1]:
        return True
    else:
        return False


def compare(f1, f2, sample):
    # 讀取兩個檔案
    try:
        with open(f1, "r") as file1:
            a = [line.strip() for line in file1 if line.strip()]
        with open(f2, "r") as file2:
            b = [line.strip() for line in file2 if line.strip()]
    except FileNotFoundError:
        print(f"檔案不存在：{f1} 或 {f2}")
        return False

    # 若有任一檔案為空，直接判定不同
    if not a or not b:
        return False

    # 兩邊都很短（1-2 行），直接比對整體內容
    if len(a) <= 2 and len(b) <= 2:
        return a == b

    # 兩邊都有多行，直接比對整體內容
    if len(a) > 2 and len(b) > 2:
        return a == b

    # ---- 以下為不對稱比對情況 ----

    # 把 a 設為短的那一份，b 為長的（方便處理）
    if len(a) > len(b):
        a, b = b, a  # swap

    # 若 a 有 2 行，先刪掉標題行
    if len(a) == 2:
        data1 = a[1]
    else:
        data1 = a[0]

    # 移除 b 的標題行
    b = b[1:]

    # 在 b 裡搜尋 data1 對應到的 sample 名稱
    for line in b:
        if data1 in line:
            parts = line.split(":")
            if parts[0] == sample:
                return True

    return False


# def compare(f1,f2,sample):
#     with open(f1,"r") as a:
#         a = a.readlines()
#     with open(f2,"r") as b:
#         b = b.readlines()
#     if (len(a) <= 2) and (len(b) <=2 ):
#         return (a == b)
#     elif (len(a) > 2) and (len(b) > 2 ):
#         return (a == b)
#     elif (len(a) <= 2) and (len(b) > 2 ):
#         if len(a) == 2:
#             del a[0]
#             data1 = a[0].strip()
#             # print(data1)
#         else:
#             data1 = a[0].strip()
#             # print(data1)
#         del b[0]
        
#         for i,v in enumerate(b):
#             # print(v.split(":")[0])
#             # print(v.split(":")[1])
#             if data1 in v.strip():
#                 if v.split(":")[0] == sample:
#                     return True
#         return False
#     elif (len(a) > 2) and (len(b) < 2 ):
#         if len(b) == 2:
#             del b[0]
#             data1 = b[0].strip()
#         else:
#             data1 = b[0].strip()
#         del a[0]
#         for i,v in enumerate(a):
#             if data1 in v.strip():
#                 if v.split(":")[0] == sample:
#                     return True
#         return False
def checkFileNum(dirpath):
    folder = Path(dirpath) 
    file_count = sum(1 for f in folder.iterdir() if f.is_file())
    return file_count

def creatName(BC, J, D, L, P, m, phys):
    mySourceName = sourcelist[phys].replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m) 
    myTargetName = tarlist[phys].replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m) 
    groupSourceName = grouplist[phys].replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m) 
    groupTargetName = tarlist[phys].replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m) 
    return (mySourceName, groupSourceName, myTargetName, groupTargetName)

def creatCpName(BC, J, D, L, P, m, phys):
    CpName = newlist[phys].replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m) 
    return CpName
def creatColName(BC, J, D, L, P, m, phys):
    colName = collist[phys].replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m) 
    return colName
def creatDir(BC, J, D, L, P, m, phys):
    sourcePath = "/".join(["tSDRG","Main_15","data_tar","BC_re","J_re","D_re","L_re_P_re_m_re_s_re"])
    tarPath = "/".join(["tSDRG","Main_15","data_collect_old","BC_re","J_re","D_re","L_re_P_re_m_re"])
    mySourcePathBase = "/".join([tSDRG_path,sourcePath])
    groupSourcePathBase = "/".join([group_path,sourcePath])
    myTargetPathBase = "/".join([tSDRG_path,tarPath])
    groupTargetPathBase = "/".join([group_path,tarPath])
    # sourcePathBase = f"{tSDRG_path}/tSDRG/Main_15/data_random/BC_re/J_re/D_re/L_re_P_re_m_re_s_re/"
    # cpPathBase = f"{group_path}/tSDRG/Main_15/data_random/BC_re/J_re/D_re/L_re_P_re_m_re_s_re/"
    # targetPathBase = f"{group_path}/tSDRG/Main_15/data_collect/BC_re/J_re/D_re/L_re_P_re_m_re/"

    mySourcePath = mySourcePathBase.replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m)
    groupSourcePath = groupSourcePathBase.replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m)
    myTargetPath = myTargetPathBase.replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m)
    groupTargetPath = groupTargetPathBase.replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m)

    # cpPath = cpPathBase.replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m)
    # targetPath = targetPathBase.replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m)
    
    return (mySourcePath, groupSourcePath, myTargetPath, groupTargetPath)

def fread(f, phys):
    if os.path.exists(f):
        with open(f,"r") as a:
            a = a.readlines()
            if len(a) == 0:
                return 
            else:
                if phys in a[0].strip():
                    del a[0]
                a = "".join(a)
                a = a.replace("\n"," ")
                return a
    else:
        return 

def create_tarball_files(output_filename, file_list):
    with tarfile.open(output_filename, "w:gz") as tar:
        for file in file_list:
            tar.add(file, arcname=file)  # arcname 保持原始檔名
    print(f"已打包 {len(file_list)} 個檔案到 {output_filename}")

def kill_files(file_list):
    for i,f in enumerate(file_list):
        os.system("rm " + f)
    return f"已刪除 {len(file_list)} 個檔案，從{file_list[0]}到{file_list[-1]}"
    
def cp_files(file_list):
    for i,f in enumerate(file_list):
        os.system("cp " + f)
    print(f"已複製 {len(file_list)} 個檔案，從{file_list[0]}到{file_list[-1]}")



def parse_context(context):
    """
    將原始字串解析為鍵值對列表。
    """
    lines = [line.strip() for line in context.strip().split('\n') if line.strip()]
    pairs = []
    for line in lines:
        if ':' in line:
            key_value = line.split(':', 1)
            if len(key_value) == 2:
                key_str, value = key_value
                try:
                    key_int = int(key_str.strip())
                    pairs.append((key_int, value.strip()))
                except ValueError:
                    continue
    return pairs

def is_sorted(pairs):
    """
    檢查鍵值對列表是否已按鍵的升序排序。
    """
    return all(pairs[i][0] <= pairs[i + 1][0] for i in range(len(pairs) - 1))

def sort_context(pairs):
    """
    對鍵值對列表按鍵進行排序，並重建為字串格式。
    """
    sorted_pairs = sorted(pairs, key=lambda x: x[0])
    s1 = sorted_pairs[0][0]  # 假設第一個鍵是 s1
    sorted_lines = [f"{key}:{value}" for key, value in sorted_pairs]
    return '\n'.join(sorted_lines), s1

def sort_if_needed(context):
    """
    若資料未排序，則進行排序；否則返回原始資料。
    """
    pairs = parse_context(context)
    if is_sorted(pairs):
        print("資料已排序，無需排序。")
        s1 = int(pairs[0][0])  # 假設第一個鍵是 s1
        return context, s1
    else:
        print("資料未排序，開始排序。")
        return sort_context(pairs)
        
def Combine(BC, J, D, L, P, m, phys, s1, s2):
    folder = creatDir(BC, J, D, L, P, m, phys)
    name = creatName(BC, J, D, L, P, m, phys)

    mySourcePath = folder[0] + "/" + name[0]
    groupSourcePath = folder[1] + "/" + name[1]
    myTarPath = folder[2] + "/" + name[2]
    groupTarPath = folder[3] + "/" + name[3]

    seedArray = list(range(s1, s2 + 1))
    # with open(groupTarPath, "r") as originFile:
    #     originaText = originFile.readlines()
    context = ""
    # print("originaText")
    for seed in seedArray:
        groupSource = groupSourcePath.replace("s_re", str(seed))
        mySource = mySourcePath.replace("s_re", str(seed))
        # if f"{seed}:" in originaText[seed-1]:
        #     continue
        if os.path.exists(groupSource) and os.path.exists(mySource):
            if compare(groupSource, mySource, seed):                
                fcontext = fread(mySource, phys)
            else:
                # os.remove(groupSource)
                shutil.copy(mySource, groupSource)
                fcontext = fread(groupSource, phys)
        elif os.path.exists(mySource):
            os.makedirs(os.path.dirname(groupSource), exist_ok=True)
            shutil.copy(mySource, groupSource)
            # os.remove(mySource)
            fcontext = fread(groupSource, phys)
        elif os.path.exists(groupSource):
            fcontext = fread(groupSource, phys)
        else:
            continue

        if fcontext is not None:
            context += f"{seed}:{fcontext}\n"

    if context != "":
        context, s1 = sort_if_needed(context)
        
        save_context(context, s1, groupTarPath, myTarPath, phys)
        # os.makedirs(os.path.dirname(myTarPath), exist_ok=True)


def save_context(context, s1, groupTarPath, myTarPath, phys):
    if not os.path.exists(groupTarPath):
        os.makedirs(os.path.dirname(groupTarPath), exist_ok=True)
    if s1 == 1:
        context = f"{phys}\n{context}"
        # print(f"[WRITE] groupTarPath: {groupTarPath}, myTarPath: {myTarPath}")
        with open(groupTarPath, "w") as f1:
            
            try:
                # 嘗試用非阻塞方式加鎖
                fcntl.flock(f1, fcntl.LOCK_EX | fcntl.LOCK_NB)
                print("✅ 立即取得鎖")
                print(f"檔案已鎖定 [WRITE] groupTarPath: {groupTarPath}, s1:{s1}, 目前進程 PID: {os.getpid()}")
                f1.write(context)
                fcntl.flock(f1, fcntl.LOCK_EX | fcntl.LOCK_NB)
                print("✅ 檔案已解鎖")    
            except BlockingIOError:
                print("⏳ 檔案已被鎖住，進入等待模式...")
                fcntl.flock(f1, fcntl.LOCK_EX)  # 這裡才會阻塞，等釋放
                print("✅ 最終取得鎖")
                print(f"檔案已鎖定 [WRITE] groupTarPath: {groupTarPath}, s1:{s1}, 目前進程 PID: {os.getpid()}")
                f1.write(context)
                fcntl.flock(f1, fcntl.LOCK_UN)
                print("✅ 檔案已解鎖")            # f2.write(context)
        # with open(groupTarPath, "w") as f1, open(myTarPath, "w") as f2:
        #     f1.write(context)
        #     # f2.write(context)
    else:
        # print(f"[APPEND] groupTarPath: {groupTarPath}, myTarPath: {myTarPath}")
        with open(groupTarPath, "a") as f1:
            
            try:
                # 嘗試用非阻塞方式加鎖
                fcntl.flock(f1, fcntl.LOCK_EX | fcntl.LOCK_NB)
                print("✅ 立即取得鎖")
                print(f"檔案已鎖定 [APPEND] groupTarPath: {groupTarPath}, s1:{s1}, 目前進程 PID: {os.getpid()}")
                f1.write(context)
                fcntl.flock(f1, fcntl.LOCK_EX | fcntl.LOCK_NB)
                print("✅ 檔案已解鎖")    
            except BlockingIOError:
                print("⏳ 檔案已被鎖住，進入等待模式...")
                fcntl.flock(f1, fcntl.LOCK_EX)  # 這裡才會阻塞，等釋放
                print("✅ 最終取得鎖")
                print(f"檔案已鎖定 [APPEND] groupTarPath: {groupTarPath}, s1:{s1}, 目前進程 PID: {os.getpid()}")
                f1.write(context)
                fcntl.flock(f1, fcntl.LOCK_UN)
                print("✅ 檔案已解鎖") 
    # # print("originaText222")
    # if s1 == 1:
    #     context = f"{phys}\n{context}"
    #     print(f"[WRITE] groupTarPath: {groupTarPath}, myTarPath: {myTarPath}")
    #     with open(groupTarPath, "w") as f1:
    #         f1.write(context)
    #         # f2.write(context)
    #     # with open(groupTarPath, "w") as f1, open(myTarPath, "w") as f2:
    #     #     f1.write(context)
    #     #     # f2.write(context)
    # else:
    #     print(f"[APPEND] groupTarPath: {groupTarPath}, myTarPath: {myTarPath}")
    #     with open(groupTarPath, "a") as f1:
    #         f1.write(context)
    #             # f2.write(context)         
    #         # with open(groupTarPath, "a") as f1, open(myTarPath, "a") as f2:
    #         #     f1.write(context)
    #         #     # f2.write(context)          
def average(BC, J, D, L, P, m, phys, s1, s2):
    folder = creatDir(BC, J, D, L, P, m, phys)
    name = creatName(BC, J, D, L, P, m, phys)

    # mySourcePath = folder[0] + "/" + name[0]
    # groupSourcePath = folder[1] + "/" + name[1]
    myTarPath = folder[2] + "/" + name[2]
    groupTarPath = folder[3] + "/" + name[3]
    
    with open(myTarPath,"r") as a:
        a = a.readlines()
        if phys in a[0].strip():
            del a[0]
        metaContext = {}
        for s in a:
            s = s.strip()
            sNum = int(s[0].split(":")[0])
            if sNum not in metaContext:
                metaContext[sNum] = {}
                
            del s[0]
            s = s.replace(" ")
            del s[0]
            for corr in s:
                if int(corr[1]) - int(corr[0]) not in dic:
                    metaContext[sNum][int(corr[1]) - int(corr[0])] = [float(corr[2])]
                else:
                    metaContext[sNum][int(corr[1]) - int(corr[0])].append(float(corr[2]))

        
def parameter_read_dict(filename):
    parameters = {}
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            for line in file:
                if ':' in line:
                    key, value = line.split(':', 1)
                    key = key.strip()
                    value = value.strip()
                    if key:
                        parameters[key] = value
    except FileNotFoundError:
        print(f"無法開啟檔案: {filename}")
    
    return parameters


            
if __name__ == "__main__":
    file = sys.argv[1]
    arg = []
    # Jstr = [f"Jdis{str(i).zfill(3)}" for i in range(int(J),int(J)+1)]
    Jstr = [f"Jdis{str(i).zfill(3)}" for i in range(30,31,50)]

    Dstr = [f"Dim{str(i).zfill(3)}" for i in range(101)]
    Lstr = [f"L{num}" for num in range(31, 255, 32)]  # 只有 L512
    Lstr = [f"L{num}" for num in range(64, 550, 64)]  # 只有 L512
    # a = scriptCreator.para("read",file)
    # parameterlist = a.para
    # para=scriptCreator.paraList1(parameterlist["L"],parameterlist["J"],parameterlist["D"],parameterlist["S"])
    # BC = parameterlist["BC"]
    # Pdis = parameterlist["Pdis"]
    # chi = "m" + str(parameterlist["chi"])
    # s1 = int(parameterlist["S"]["S1"])
    # s2 = int(parameterlist["S"]["S2"])
    # s1 = int(sys.argv[2])
    # s2 = int(sys.argv[3])
    BC = "PBC"
    Pdis = 10
    chi = "m40"
    if BC == "PBC":
        s_list = ["ZL","corr1","corr2","string","J_list","energy","dimerization","w_loc","seed"]
        s_list = ["ZL"]
    else:
        s_list = ["ZL","corr1","corr2","J_list","energy","dimerization","w_loc","seed"]
        
    # for s in s_list:
    #     for L in para.L_str:
    #         for J in para.J_str:
    #                 arg.append((BC, J, para.D_str[0], L, f"P{Pdis}", f"{chi}", s, s1, s2))
    s1 = 1
    s2 = 10000  
    for s in s_list:
        for L in Lstr:
            for J in Jstr:
                    arg.append((BC, J, Dstr[0], L, f"P{Pdis}", f"{chi}", s, s1, s2))
    print(arg)         

    def fun(arg):
        print("---------------------col--------------------\n")
        with multiprocessing.Pool(processes=10) as pool:
            results1 = pool.starmap(Combine, arg)
        # print("---------------------del--------------------\n")
        # with multiprocessing.Pool(processes=20) as pool:
        #     results1 = pool.starmap(checkAndDelete, arg)
            
    # 計算函數執行時間
    execution_time = timeit.timeit(lambda: fun(arg), number=1)

    # 執行並顯示結果
    # results1, results2 = fun(arg)
    print(f"Execution time: {execution_time} seconds")    


[('PBC', 'Jdis030', 'Dim000', 'L64', 'P10', 'm40', 'ZL', 1, 10000), ('PBC', 'Jdis030', 'Dim000', 'L128', 'P10', 'm40', 'ZL', 1, 10000), ('PBC', 'Jdis030', 'Dim000', 'L192', 'P10', 'm40', 'ZL', 1, 10000), ('PBC', 'Jdis030', 'Dim000', 'L256', 'P10', 'm40', 'ZL', 1, 10000), ('PBC', 'Jdis030', 'Dim000', 'L320', 'P10', 'm40', 'ZL', 1, 10000), ('PBC', 'Jdis030', 'Dim000', 'L384', 'P10', 'm40', 'ZL', 1, 10000), ('PBC', 'Jdis030', 'Dim000', 'L448', 'P10', 'm40', 'ZL', 1, 10000), ('PBC', 'Jdis030', 'Dim000', 'L512', 'P10', 'm40', 'ZL', 1, 10000)]
---------------------col--------------------

資料已排序，無需排序。
✅ 立即取得鎖
檔案已鎖定 [WRITE] groupTarPath: /home/aronton/tSDRG_random/tSDRG/Main_15/data_collect_old/PBC/Jdis030/Dim000/L64_P10_m40/ZL_L64_P10_m40_Jdis030_Dim000.txt, s1:1, 目前進程 PID: 2711533
✅ 檔案已解鎖
資料已排序，無需排序。
✅ 立即取得鎖
檔案已鎖定 [WRITE] groupTarPath: /home/aronton/tSDRG_random/tSDRG/Main_15/data_collect_old/PBC/Jdis030/Dim000/L256_P10_m40/ZL_L256_P10_m40_Jdis030_Dim000.txt, s1:1, 目前進程 PID: 2711536
✅ 檔案已解鎖


In [ ]:
import os
import math
import time
import timeit
import sys
import tarfile
import datetime
import multiprocessing
import scriptCreator
from pathlib import Path
import shutil
import numpy as np



dicosPath = "/ceph/work/NTHU-qubit/LYT/tSDRG_random"
scopionPath = "/home/aronton/tSDRG_random"

if os.path.isdir(dicosPath):
    tSDRG_path = dicosPath
    group_path = dicosPath
    
if os.path.isdir(scopionPath):
    tSDRG_path = scopionPath
    group_path = scopionPath
    
sourcelist = {"ZL":"ZL.csv", "energy":"energy.csv", "seed":"s_re_seed.csv",\
    "corr1":"_".join(["L_re","P_re","m_re","s_re","corr1.csv"]), "corr2":"_".join(["L_re","P_re","m_re","s_re","corr2.csv"]),\
    "ZLI":"ZLI.csv", "ZLC":"ZLC.csv", "w_loc":"w_loc.csv", "J_list":"J_list.csv", "dimerization":"dimerization.csv",\
    "string":"_".join(["L_re","P_re","m_re","s_re","string.csv"])
    }

grouplist = {"ZL":"_".join(["L_re","P_re","m_re","s_re","ZL.txt"]), "energy":"_".join(["L_re","P_re","m_re","s_re","energy.txt"]),\
    "corr1":"_".join(["L_re","P_re","m_re","s_re","corr1.txt"]), "corr2":"_".join(["L_re","P_re","m_re","s_re","corr2.txt"]),\
    "ZLI":"_".join(["L_re","P_re","m_re","s_re","ZLI.txt"]), "ZLC":"_".join(["L_re","P_re","m_re","s_re","ZLC.txt"]),\
    "w_loc":"_".join(["L_re","P_re","m_re","s_re","w_loc.txt"]), "J_list":"_".join(["L_re","P_re","m_re","s_re","J_list.txt"]),\
    "string":"_".join(["L_re","P_re","m_re","s_re","string.txt"]), "seed":"_".join(["L_re","P_re","m_re","s_re","seed.txt"]),\
    "dimerization":"_".join(["L_re","P_re","m_re","s_re","dimerization.txt"])
    }

tarlist = {
    "ZL":"_".join(["ZL","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "energy":"_".join(["energy","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "corr1":"_".join(["corr1","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "corr2":"_".join(["corr2","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "ZLI":"_".join(["ZLI","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "ZLC":"_".join(["ZLC","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "w_loc":"_".join(["w_loc","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "J_list":"_".join(["J_list","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "string":"_".join(["string","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "seed":"_".join(["seed","L_re","P_re","m_re.txt"]),
    "dimerization":"_".join(["dimerization","L_re","P_re","m_re.txt"])
    }

metalist = {
    "ZL":"_".join(["ZL","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "gap":"_".join(["energy","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "corr1":"_".join(["corr1","L_re","P_re","m_re","J_re","D_re","dx_re.txt"]),\
    "corr2":"_".join(["corr2","L_re","P_re","m_re","J_re","D_re","dx_re.txt"]),\
    }

metaDislist = {
    "ZL":"_".join(["ZL_dis","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "gap":"_".join(["gap_dis","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "corr1":"_".join(["corr1_dis","L_re","P_re","m_re","J_re","D_re","dx_re.txt"]),\
    "corr2":"_".join(["corr2_dis","L_re","P_re","m_re","J_re","D_re","dx_re.txt"]),\
    }


def creatName(BC, J, D, L, P, m, phys):
    mySourceName = sourcelist[phys].replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m) 
    myTargetName = tarlist[phys].replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m) 
    groupSourceName = grouplist[phys].replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m) 
    groupTargetName = tarlist[phys].replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m) 
    groupAveName = metalist[phys].replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m)
    groupDisName = metaDislist[phys].replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m)
    return (mySourceName, groupSourceName, myTargetName, groupTargetName, groupAveName, groupDisName)

# def creatCpName(BC, J, D, L, P, m, phys):
#     CpName = newlist[phys].replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m) 
#     return CpName
# def creatColName(BC, J, D, L, P, m, phys):
#     colName = collist[phys].replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m) 
#     return colName
def creatDir(BC, J, D, L, P, m, phys):
    avePath = "/".join(["tSDRG","Main_15","metadata_old","BC_re","J_re","D_re","L_re_P_re_m_re_s_re"])
    sourcePath = "/".join(["tSDRG","Main_15","data_tar","BC_re","J_re","D_re","L_re_P_re_m_re_s_re"])
    tarPath = "/".join(["tSDRG","Main_15","data_collect_old","BC_re","J_re","D_re","L_re_P_re_m_re"])
    mySourcePathBase = "/".join([tSDRG_path,sourcePath])
    groupSourcePathBase = "/".join([group_path,sourcePath])
    myTargetPathBase = "/".join([tSDRG_path,tarPath])
    groupTargetPathBase = "/".join([group_path,tarPath])
    avePathBase = "/".join([tSDRG_path,avePath])
    # disPathBase = "/".join([tSDRG_path,avePath])
    # sourcePathBase = f"{tSDRG_path}/tSDRG/Main_15/data_random/BC_re/J_re/D_re/L_re_P_re_m_re_s_re/"
    # cpPathBase = f"{group_path}/tSDRG/Main_15/data_random/BC_re/J_re/D_re/L_re_P_re_m_re_s_re/"
    # targetPathBase = f"{group_path}/tSDRG/Main_15/data_collect/BC_re/J_re/D_re/L_re_P_re_m_re/"

    mySourcePath = mySourcePathBase.replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m)
    groupSourcePath = groupSourcePathBase.replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m)
    myTargetPath = myTargetPathBase.replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m)
    groupTargetPath = groupTargetPathBase.replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m)
    myAvePath = avePathBase.replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m).replace("s_re", "meta")
    myDisPath = avePathBase.replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m).replace("s_re", "dis")
    # cpPath = cpPathBase.replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m)
    # targetPath = targetPathBase.replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m)
    
    return (mySourcePath, groupSourcePath, myTargetPath, groupTargetPath, myAvePath, myDisPath)

def gapAverage(BC, J, D, L, P, m, phys):
    folder = creatDir(BC, J, D, L, P, m, phys)
    name = creatName(BC, J, D, L, P, m, phys)
    myTarPath = folder[2] + "/" + name[2]

    gaplist = []
    try:
        with open(myTarPath, "r") as targetFile:
            collect = targetFile.readlines()
            collect = [line.strip() for line in collect]
            if collect and collect[0] == "energy":
                del collect[0]
            for line in collect:
                line = line.split(":")[-1].split(" ")
                try:
                    gap = float(line[1]) - float(line[0])
                    gaplist.append(gap)
                except Exception as e:
                    print(e)
    except FileNotFoundError:
        print(f"File not found: {myTarPath}")
        return False, False, False
    save_gapDistribute(gaplist, BC, J, D, L, P, m, phys)
    gapAve = np.mean(gaplist)
    sample = len(gaplist) 
    error = np.std(gaplist, ddof=1)

    return gapAve, sample, error

def save_gapDistribute(gaplist, BC, J, D, L, P, m, phys):
    folder = creatDir(BC, J, D, L, P, m, phys)
    name = creatName(BC, J, D, L, P, m, phys)
    # print( f"folder[4]:{folder[4]}")
    gapDisBase = folder[5] + "/" + name[5]
    context = "ground_state_gap\n"
    for i, value in enumerate(gaplist):
        context += f"{value}\n"
    
    if context == "ground_state_gap\n":
        return
    else:
        if not os.path.exists(gapDisBase):
            os.makedirs(os.path.dirname(gapDisBase), exist_ok=True)
        with open(gapDisBase, "w") as targetFile:
            targetFile.write(context)

def save_gap(BC, J, D, L, P, m, phys):
    gapAve, sample, error = gapAverage(BC, J, D, L, P, m, phys)
    if gapAve == False:
        print(f"Error: No data found for {BC}, {J}, {D}, {L}, {P}, {m}, {phys}")
        return
    folder = creatDir(BC, J, D, L, P, m, phys)
    name = creatName(BC, J, D, L, P, m, phys)
    # print( f"folder[4]:{folder[4]}")
    myTarPath = folder[4] + "/" + name[4]
    # print(f"myTarPath:{myTarPath}")
    if not os.path.exists(myTarPath):
        os.makedirs(os.path.dirname(myTarPath), exist_ok=True)
    with open(myTarPath, "w") as targetFile:
        targetFile.write(f"ground_state_energy, sample, error\n{gapAve}, {sample}, {error/math.sqrt(sample)}")
            
def corrAverage(BC, J, D, L, P, m, phys):
    folder = creatDir(BC, J, D, L, P, m, phys)
    name = creatName(BC, J, D, L, P, m, phys)
    myTarPath = folder[2] + "/" + name[2]

    corrDic = {}
    try:
        with open(myTarPath, "r") as targetFile:
            collect = targetFile.readlines()
            collect = [line.strip() for line in collect]
            if collect and collect[0] == "corr1":
                del collect[0]
            for line in collect:
                line = line.split(":")[-1].split(" ")
                for data in line:
                    if data:
                        parts = data.split(",")
                        if len(parts) >= 3:
                            try:
                                x1, x2 = int(parts[0]), int(parts[1])
                                corr = float(parts[2])
                                key = x2 - x1
                                if key not in corrDic:
                                    corrDic[key] = []
                                corrDic[key].append(corr)
                            except ValueError:
                                continue  # 忽略無法轉換的數據
    except FileNotFoundError:
        print(f"File not found: {myTarPath}")
        return False, False, False
    save_corrDistribute(corrDic, BC, J, D, L, P, m, phys)
    corr = {}
    sample = {}
    error = {}
    for key, values in corrDic.items():
        sample[key] = len(values)
        corr[key] = np.mean(values)
        error[key] = np.std(values, ddof=1) if len(values) > 1 else 0.0

    return corr, sample, error

def save_corrDistribute(corrDic, BC, J, D, L, P, m, phys):
    folder = creatDir(BC, J, D, L, P, m, phys)
    name = creatName(BC, J, D, L, P, m, phys)
    # print( f"folder[4]:{folder[4]}")
    corrDisBase = folder[5] + "/" + name[5]
    for key, values in corrDic.items():
        context = ""
        if BC == "OBC":
            context = f"C_etoe=<S(0)S(dx={key})>\n"
        elif BC == "PBC":
            context = f"C_bulk=<S(0)S(dx={key})>\n"
        corrDisPath = corrDisBase.replace("dx_re", f"dx={key}")
        for value in values:
            context += f"{value}\n"
        
        if context == f"C_etoe=<S(0)S(dx={key})>\n" or context ==  f"C_bulk=<S(0)S(dx={key})>\n":
            continue
        else:
            if not os.path.exists(corrDisPath):
                os.makedirs(os.path.dirname(corrDisPath), exist_ok=True)
            with open(corrDisPath, "w") as targetFile:
                targetFile.write(context)

def save_corr(BC, J, D, L, P, m, phys):
    corr, sample, error = corrAverage(BC, J, D, L, P, m, phys)
    if corr == False:
        print(f"Error: No data found for {BC}, {J}, {D}, {L}, {P}, {m}, {phys}")
        return
    folder = creatDir(BC, J, D, L, P, m, phys)
    name = creatName(BC, J, D, L, P, m, phys)
    # print( f"folder[4]:{folder[4]}")
    myTarPathBase = folder[4] + "/" + name[4]
    # print(f"myTarPath:{myTarPath}")

    for key in corr.keys():
        if corr[key] == None:
            continue
        myTarPath = myTarPathBase.replace("dx_re", f"dx={key}")
        if BC == "OBC":
            context = f"C_etoe=<S(0)S(dx={key})>,sample,errorbar\n" + f"{corr[key]},{sample[key]},{error[key]/math.sqrt(sample[key])}"
        elif BC == "PBC":
            context = f"C_bulk=<S(0)S(dx={key})>,sample,errorbar\n" + f"{corr[key]},{sample[key]},{error[key]/math.sqrt(sample[key])}"
        if not os.path.exists(myTarPath):
            os.makedirs(os.path.dirname(myTarPath), exist_ok=True)
        with open(myTarPath, "w") as targetFile:
            targetFile.write(context)


def ZLAverage(BC, J, D, L, P, m, phys):
    folder = creatDir(BC, J, D, L, P, m, phys)
    name = creatName(BC, J, D, L, P, m, phys)
    myTarPath = folder[2] + "/" + name[2]
    zllist = []
    
    try:
        if not os.path.exists(myTarPath):
            raise FileNotFoundError(f"{myTarPath} does not exist")
        auto_delete_empty = True
        if os.path.getsize(myTarPath) == 0:
            if auto_delete_empty:
                os.remove(myTarPath)
                print(f"[刪除] 空檔案已刪除：{myTarPath}")
            raise ValueError("檔案為空")
        with open(myTarPath, "r") as targetFile:
            zllist = targetFile.readlines()
            if type(zllist[0]) == str or zllist[0] == "ZL":
                del zllist[0]
            zllist = [float(line.split(":")[-1]) for line in zllist]
        if not zllist:
            if auto_delete_empty:
                os.remove(myTarPath)
                print(f"[刪除] 空檔案已刪除：{myTarPath}")
            raise ValueError("檔案內容為空")

    except FileNotFoundError:
        print(f"File not found: {myTarPath}")
        return False, False, False
    
    save_zlDistribute(zllist, BC, J, D, L, P, m, phys)
    zlAve = np.mean(zllist)
    sample = len(zllist)
    error = np.std(zllist, ddof=1)

    return zlAve, sample, error

def save_zlDistribute(zllist, BC, J, D, L, P, m, phys):
    folder = creatDir(BC, J, D, L, P, m, phys)
    name = creatName(BC, J, D, L, P, m, phys)
    # print( f"folder[4]:{folder[4]}")
    zlDisBase = folder[5] + "/" + name[5]
    context = "ZL\n"
    for i, value in enumerate(zllist):
        context += f"{value}\n"
    if context == "ZL\n":
        return
    else:
        if not os.path.exists(zlDisBase):
            os.makedirs(os.path.dirname(zlDisBase), exist_ok=True)
        with open(zlDisBase, "w") as targetFile:
            targetFile.write(context)

def save_ZL(BC, J, D, L, P, m, phys):
    zlAve, sample, error = ZLAverage(BC, J, D, L, P, m, phys)
    if zlAve == False:
        print(f"Error: No data found for {BC}, {J}, {D}, {L}, {P}, {m}, {phys}")
        return
    folder = creatDir(BC, J, D, L, P, m, phys)
    name = creatName(BC, J, D, L, P, m, phys)
    # print( f"folder[4]:{folder[4]}")
    myTarPath = folder[4] + "/" + name[4]

    if not os.path.exists(myTarPath):
        os.makedirs(os.path.dirname(myTarPath), exist_ok=True)
    with open(myTarPath, "w") as targetFile:
        targetFile.write(f"ZL, sample, errorbar\n{zlAve}, {sample}, {error/math.sqrt(sample)}")    
# def save_gap():

def list_txt_files(directory):
    txt_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.txt'):
                full_path = os.path.join(root, file)
                txt_files.append(full_path)
    return txt_files

if __name__ == "__main__":
    # file = sys.argv[1]
    # arg = []
    # a = scriptCreator.para("read",file)
    # parameterlist = a.para
    # para=scriptCreator.paraList1(parameterlist["L"],parameterlist["J"],parameterlist["D"],parameterlist["S"])
    # BC = parameterlist["BC"]
    # Pdis = parameterlist["Pdis"]
    # chi = "m" + str(parameterlist["chi"])
    # s1 = int(sys.argv[2])
    # s2 = int(sys.argv[3])
    # if BC == "PBC":
    #     s_list = ["ZL","corr1","corr2","string","J_list","energy","dimerization","w_loc","seed"]
    #     s_list = ["corr1","corr2"]
    # else:
    #     s_list = ["ZL","corr1","corr2","J_list","energy","dimerization","w_loc","seed"]
    #     s_list = ["corr1","corr2"]
        
    # # for s in s_list:
    # for D in para.D_str:
    #     for L in para.L_str:
    #         for J in para.J_str:
    #             # save_corr(BC, J, D, L, f"P{Pdis}", f"{chi}", "corr1")    
    #             # save_corr(BC, J, D, L, f"P{Pdis}", f"{chi}", "corr2")
    #             # save_gap(BC, J, D, L, f"P{Pdis}", f"{chi}", "energy")   
    #             save_ZL(BC, J, D, L, f"P{Pdis}", f"{chi}", "ZL")   
    #                 # arg.append((BC, J, para.D_str[0], L, f"P{Pdis}", f"{chi}", s, s1, s2))
    # Jstr = [f"Jdis{str(i).zfill(3)}" for i in range(int(J),int(J)+1)]
             
    # 參數設定
    D_i, D_f, D_d = 0, 61, 1
    J_i, J_f, J_d = 1, 51, 1
    L_i, L_f, L_d = 32, 550, 32                    
        
    D_list = [f"Dim{str(i).zfill(3)}" for i in range(D_i, D_f, D_d)]
    J_list = [f"Jdis{str(i).zfill(3)}" for i in range(J_i, J_f, J_d)]
    L_list = [f"L{num}" for num in range(L_i, L_f, L_d)]
    BC = "PBC"
    Pdis = 10
    chi = "m" + str(40)
    s1 = 1
    s2 = 10000
    for D in D_list:
        for L in L_list:
            for J in J_list:
                # save_corr(BC, J, D, L, f"P{Pdis}", f"{chi}", "corr1")    
                # save_corr(BC, J, D, L, f"P{Pdis}", f"{chi}", "corr2")
                # save_gap(BC, J, D, L, f"P{Pdis}", f"{chi}", "energy")   
                save_ZL(BC, J, D, L, f"P{Pdis}", f"{chi}", "ZL")   
    # save_corr(BC, J, para.D_str[0], L, f"P{Pdis}", f"{chi}", "corr1")    
    # save_corr(BC, J, para.D_str[0], L, f"P{Pdis}", f"{chi}", "corr2")
    # save_gap(BC, J, para.D_str[0], L, f"P{Pdis}", f"{chi}", "energy")   
    # save_ZL(BC, J, para.D_str[0], L, f"P{Pdis}", f"{chi}", "ZL")   


-0.0003649455223832696 2000 0.2653707850853128
0.003191211143744681 5000 0.3512899960553954
File not found: /home/aronton/tSDRG_random/tSDRG/Main_15/data_collect_old/PBC/Jdis030/Dim000/L192_P10_m40/ZL_L192_P10_m40_Jdis030_Dim000.txt
Error: No data found for PBC, Jdis030, Dim000, L192, P10, m40, ZL
0.004632788702475768 5000 0.42475869585796416
File not found: /home/aronton/tSDRG_random/tSDRG/Main_15/data_collect_old/PBC/Jdis030/Dim000/L320_P10_m40/ZL_L320_P10_m40_Jdis030_Dim000.txt
Error: No data found for PBC, Jdis030, Dim000, L320, P10, m40, ZL
-0.008319057236040553 5000 0.4652774721489594
File not found: /home/aronton/tSDRG_random/tSDRG/Main_15/data_collect_old/PBC/Jdis030/Dim000/L448_P10_m40/ZL_L448_P10_m40_Jdis030_Dim000.txt
Error: No data found for PBC, Jdis030, Dim000, L448, P10, m40, ZL
-0.0082047040317973 5000 0.4766786822883327


In [ ]:
import os
import math
import time
import timeit
import sys
import tarfile
import datetime
from multiprocessing import Pool
import scriptCreator
from pathlib import Path
import shutil

def list_leaf_dirs(root_dir):
    leaves = []
    for dirpath, dirnames, filenames in os.walk(root_dir):
        if not dirnames:
            leaves.append(dirpath)
        # print(dirpath)
    return leaves

def copy_folder(args):
    Jstr = args
    # print("args:",Jstr)
    root = f"/homeold/aronton/tSDRG_random/tSDRG/Main_15/data_random/OBC/{Jstr}/Dim000/"
    for leaf in list_leaf_dirs(root):
        leaf1 = leaf.replace('homeold', 'home').replace('data_random', 'data_random_old')
        print("leaf:", leaf)
        shutil.copytree(leaf, leaf1, dirs_exist_ok=True)

if __name__ == "__main__":
    Jlist = ["Jdis010", "Jdis020", "Jdis030", "Jdis040", "Jdis050", "Jdis060", "Jdis070", "Jdis080", "Jdis090", "Jdis100",
     "Jdis110", "Jdis120", "Jdis130", "Jdis140", "Jdis150", "Jdis160", "Jdis170", "Jdis180", "Jdis190", "Jdis200"]
    Jlist = ["Jdis050", "Jdis060"]
    def fun(arg):
        with Pool() as pool:
            pool.map(copy_folder, arg) 
        
    # 計算函數執行時間
    execution_time = timeit.timeit(lambda: fun(Jlist), number=1)

    # 執行並顯示結果
    # results1, results2 = fun(arg)
    print(f"Execution time: {execution_time} seconds")

In [ ]:
import os

def check(BC, J, D, L, P, m, phys, path):

    path / BC / J / D / L / P / m / 



base_path = "/home/aronton/tSDRG_random/tSDRG/Main_15/data_random/PBC/Jdis010/Dim002"


In [6]:
import os

def collect_matching_paths(base_dir, output_file):
    target_tags = [f"_{i}" for i in range(1000, 10001, 1000)]
    records = []

    for root, dirs, files in os.walk(base_dir):
        for name in dirs + files:
            for tag in target_tags:
                if tag in name:
                    val = int(tag[1:])
                    full_path = os.path.join(root, name)
                    records.append((val, full_path))
                    break  # 一旦匹配其中一個 tag 就跳出

    # 依數字大小排序（可省略）
    records.sort(key=lambda x: x[0])

    # 寫入文字檔
    with open(output_file, 'w') as f:
        for val, path in records:
            f.write(f"{val},{path}\n")

    print(f"共儲存 {len(records)} 筆結果到 {output_file}")

# 📂 設定資料夾與輸出檔
base_path = "/home/aronton/tSDRG_random/tSDRG/Main_15/data_random/PBC/Jdis010/Dim002"
output_path = "matched_paths.csv"  # 存在執行目錄

# 🚀 執行
collect_matching_paths(base_path, output_path)

共儲存 200 筆結果到 matched_paths.csv


In [4]:
import os

def find_max_path_by_suffixes(base_dir):
    # 建立 _1000 到 _10000 的字串列表
    target_tags = [f"_{i}" for i in range(1000, 10001, 1000)]

    max_val = -1
    max_path = None

    for root, dirs, files in os.walk(base_dir):
        for name in dirs + files:  # 同時看資料夾與檔案
            for tag in target_tags:
                if tag in name:
                    value = int(tag[1:])  # 去掉 "_" 轉成 int
                    if value > max_val:
                        max_val = value
                        max_path = os.path.join(root, name)
                    break  # 找到就跳出，避免多次匹配

    return max_path, max_val

# 📁 指定資料夾路徑
base_path = "/home/aronton/tSDRG_random/tSDRG/Main_15/data_random/PBC/Jdis010/Dim002"

# 🔍 執行
path, val = find_max_path_by_suffixes(base_path)

# 🖨️ 結果
if path:
    print(f"找到最大目標：{path}（數字：{val}）")
else:
    print("找不到 _1000 到 _10000 的匹配項目")

找到最大目標：/home/aronton/tSDRG_random/tSDRG/Main_15/data_random/PBC/Jdis010/Dim002/L16_P10_m40_5000（數字：5000）


In [ ]:
def check(BC, J, D, L, P, m, phys, path):
    folder = folder_re.replace("L_re", L).replace("P_re", f"P{P}").replace("chi_re", m)
    name = name_re.replace("L_re", L).replace("J_re", J).replace("P_re", f"P{P}").replace("chi_re", m).replace("J_re", J).replace("D_re", D).replace("dx_re", f"dx={L_num[l]-1}")
    path = Path(folder, J_list[j], D_list[d], name)  

BC = 

In [71]:
a = list(range(1000, 10001, 1000))
a.reverse()
print(a)

[10000, 9000, 8000, 7000, 6000, 5000, 4000, 3000, 2000, 1000]


In [69]:
a=list(range(1000,10000,1000))
print(reverse(a))

NameError: name 'reverse' is not defined

In [64]:
a.reverse()

In [55]:

for i in range(10000,1000,1000):
    print(i)

In [81]:
def check(BC, J, D, L, P, m, phys, path):
    folder = folder_re.replace("L_re", L).replace("P_re", f"P{P}").replace("chi_re", m)
    name = creatName(BC, "Jdis010", "Dim002", L, f"P{P}", "m40", "corr1")[1].replace("s_re", "1000")
    # name = name_re.replace("L_re", L).replace("J_re", J).replace("P_re", f"P{P}").replace("chi_re", m).replace("J_re", J).replace("D_re", D).replace("dx_re", f"dx={L_num[l]-1}")
    path = Path(folder, name)
    print(path)
check("PBC", "Jdis010", "Dim002", "L32", "10", "m40", "corr1", "/home/aronton/tSDRG_random/tSDRG/Main_15/data_random")

/home/aronton/tSDRG_random/tSDRG/Main_15/data_random/PBC/Jdis010/Dim002/L32_P10_m40_s_re/L32_P10_m40_1000_corr1.txt


In [48]:
BC = "PBC"
J = "Jdis010" 
D = "Dim002"
L = "L32"
P = 10
m = "m40"
phys = "corr1"

folder_re = creatDir(BC, "Jdis010", "Dim002", "L32", "P10", "m40", "corr1")[1]
folder = folder_re.replace("L_re", L).replace("P_re", "P10").replace("chi_re", m).replace("s_re", "1000")
print(folder)
name = creatName(BC, "Jdis010", "Dim002", L, f"P{P}", "m40", "corr1")[1].replace("s_re", "1000")
print(name)
# name = name_re.replace("L_re", L).replace("J_re", J).replace("P_re", f"P{P}").replace("chi_re", m).replace("J_re", J).replace("D_re", D).replace("dx_re", f"dx=10")
path = Path(folder, name)  
print(path)

with path.open("r") as f:
    lines = f.readlines()
print(lines)
print(folder, name, path)

/home/aronton/tSDRG_random/tSDRG/Main_15/data_random/PBC/Jdis010/Dim002/L32_P10_m40_1000
L32_P10_m40_1000_corr1.txt
/home/aronton/tSDRG_random/tSDRG/Main_15/data_random/PBC/Jdis010/Dim002/L32_P10_m40_1000/L32_P10_m40_1000_corr1.txt
['x1,x2,corr\n', '0,16,0.4676337859568083\n', '1,17,0.614843786992552\n', '2,18,0.6570779590296938\n', '3,19,0.5381440064734961\n', '4,20,0.6847521533740805\n', '5,21,0.8681645920355825\n', '6,22,0.8833595849874735\n', '7,23,1.022252688834315\n', '8,24,1.002041884548506\n', '9,25,0.7811412471147365\n', '10,26,0.5905479712856296\n', '11,27,0.3992787930836768\n', '12,28,0.3380841680219221\n', '13,29,0.4532873641867693\n', '14,30,0.4618304648040701\n', '15,31,0.4085698008420554\n']
/home/aronton/tSDRG_random/tSDRG/Main_15/data_random/PBC/Jdis010/Dim002/L32_P10_m40_1000 L32_P10_m40_1000_corr1.txt /home/aronton/tSDRG_random/tSDRG/Main_15/data_random/PBC/Jdis010/Dim002/L32_P10_m40_1000/L32_P10_m40_1000_corr1.txt


In [15]:
creatDir("OBC", "Jdis010", "Dim002", "L32", "P10", "m40", "corr1")[0]

'/home/aronton/tSDRG_random/tSDRG/Main_15/data_random/OBC/Jdis010/Dim002/L32_P10_m40_s_re'

In [10]:
creatName("OBC", "Jdis010", "Dim002", "L", "P", "m", "corr1")

('L_P_m_s_re_corr1.csv',
 'L_P_m_s_re_corr1.txt',
 'corr1_L_P_m_Jdis010_Dim002.txt',
 'corr1_L_P_m_Jdis010_Dim002.txt',
 'corr1_L_P_m_Jdis010_Dim002_dx_re.txt',
 'corr1_dis_L_P_m_Jdis010_Dim002_dx_re.txt')

In [17]:
import os
import math
import time
import timeit
import sys
import tarfile
import datetime
import multiprocessing
import scriptCreator
from pathlib import Path
import shutil
import numpy as np
from pathlib import Path



tSDRG_path = "/home/aronton/tSDRG_random"
# group_path = "/ceph/work/NTHU-qubit/LYT/tSDRG_random"
group_path = "/home/aronton/tSDRG_random"
    
sourcelist = {"ZL":"ZL.csv", "energy":"energy.csv", "seed":"s_re_seed.csv",\
    "corr1":"_".join(["L_re","P_re","m_re","s_re","corr1.csv"]), "corr2":"_".join(["L_re","P_re","m_re","s_re","corr2.csv"]),\
    "ZLI":"ZLI.csv", "ZLC":"ZLC.csv", "w_loc":"w_loc.csv", "J_list":"J_list.csv", "dimerization":"dimerization.csv",\
    "string":"_".join(["L_re","P_re","m_re","s_re","string.csv"])
    }

grouplist = {"ZL":"_".join(["L_re","P_re","m_re","s_re","ZL.txt"]), "energy":"_".join(["L_re","P_re","m_re","s_re","energy.txt"]),\
    "corr1":"_".join(["L_re","P_re","m_re","s_re","corr1.txt"]), "corr2":"_".join(["L_re","P_re","m_re","s_re","corr2.txt"]),\
    "ZLI":"_".join(["L_re","P_re","m_re","s_re","ZLI.txt"]), "ZLC":"_".join(["L_re","P_re","m_re","s_re","ZLC.txt"]),\
    "w_loc":"_".join(["L_re","P_re","m_re","s_re","w_loc.txt"]), "J_list":"_".join(["L_re","P_re","m_re","s_re","J_list.txt"]),\
    "string":"_".join(["L_re","P_re","m_re","s_re","string.txt"]), "seed":"_".join(["L_re","P_re","m_re","s_re","seed.txt"]),\
    "dimerization":"_".join(["L_re","P_re","m_re","s_re","dimerization.txt"])
    }

tarlist = {
    "ZL":"_".join(["ZL","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "energy":"_".join(["energy","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "corr1":"_".join(["corr1","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "corr2":"_".join(["corr2","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "ZLI":"_".join(["ZLI","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "ZLC":"_".join(["ZLC","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "w_loc":"_".join(["w_loc","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "J_list":"_".join(["J_list","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "string":"_".join(["string","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "seed":"_".join(["seed","L_re","P_re","m_re.txt"]),
    "dimerization":"_".join(["dimerization","L_re","P_re","m_re.txt"])
    }

metalist = {
    "ZL":"_".join(["ZL","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "gap":"_".join(["energy","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "corr1":"_".join(["corr1","L_re","P_re","m_re","J_re","D_re","dx_re.txt"]),\
    "corr2":"_".join(["corr2","L_re","P_re","m_re","J_re","D_re","dx_re.txt"]),\
    }

metaDislist = {
    "ZL":"_".join(["ZL_dis","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "gap":"_".join(["gap_dis","L_re","P_re","m_re","J_re","D_re.txt"]),\
    "corr1":"_".join(["corr1_dis","L_re","P_re","m_re","J_re","D_re","dx_re.txt"]),\
    "corr2":"_".join(["corr2_dis","L_re","P_re","m_re","J_re","D_re","dx_re.txt"]),\
    }


def creatName(BC, J, D, L, P, m, phys):
    mySourceName = sourcelist[phys].replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m) 
    myTargetName = tarlist[phys].replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m) 
    groupSourceName = grouplist[phys].replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m) 
    groupTargetName = tarlist[phys].replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m) 
    groupAveName = metalist[phys].replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m)
    groupDisName = metaDislist[phys].replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m)
    return (mySourceName, groupSourceName, myTargetName, groupTargetName, groupAveName, groupDisName)

def creatCpName(BC, J, D, L, P, m, phys):
    CpName = newlist[phys].replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m) 
    return CpName
def creatColName(BC, J, D, L, P, m, phys):
    colName = collist[phys].replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m) 
    return colName
def creatDir(BC, J, D, L, P, m, phys):
    avePath = "/".join(["tSDRG","Main_15","metadata_old","BC_re","J_re","D_re","L_re_P_re_m_re_s_re"])
    sourcePath = "/".join(["tSDRG","Main_15","data_random_old","BC_re","J_re","D_re","L_re_P_re_m_re_s_re"])
    tarPath = "/".join(["tSDRG","Main_15","data_collect_old","BC_re","J_re","D_re","L_re_P_re_m_re"])
    mySourcePathBase = "/".join([tSDRG_path,sourcePath])
    groupSourcePathBase = "/".join([group_path,sourcePath])
    myTargetPathBase = "/".join([tSDRG_path,tarPath])
    groupTargetPathBase = "/".join([group_path,tarPath])
    avePathBase = "/".join([tSDRG_path,avePath])
    # disPathBase = "/".join([tSDRG_path,avePath])
    # sourcePathBase = f"{tSDRG_path}/tSDRG/Main_15/data_random/BC_re/J_re/D_re/L_re_P_re_m_re_s_re/"
    # cpPathBase = f"{group_path}/tSDRG/Main_15/data_random/BC_re/J_re/D_re/L_re_P_re_m_re_s_re/"
    # targetPathBase = f"{group_path}/tSDRG/Main_15/data_collect/BC_re/J_re/D_re/L_re_P_re_m_re/"

    mySourcePath = mySourcePathBase.replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m)
    groupSourcePath = groupSourcePathBase.replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m)
    myTargetPath = myTargetPathBase.replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m)
    groupTargetPath = groupTargetPathBase.replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m)
    myAvePath = avePathBase.replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m).replace("s_re", "meta")
    myDisPath = avePathBase.replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m).replace("s_re", "dis")
    # cpPath = cpPathBase.replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m)
    # targetPath = targetPathBase.replace("BC_re", BC).replace("J_re", J).replace("D_re", D).replace("L_re", L).replace("P_re", P).replace("m_re", m)
    
    return (mySourcePath, groupSourcePath, myTargetPath, groupTargetPath, myAvePath, myDisPath)

def gapAverage(BC, J, D, L, P, m, phys):
    folder = creatDir(BC, J, D, L, P, m, phys)
    name = creatName(BC, J, D, L, P, m, phys)
    myTarPath = folder[2] + "/" + name[2]
    gaplist = []
    with open(myTarPath, "r") as targetFile:
        collect = targetFile.readlines()
        collect = [line.strip() for line in collect]
        if collect and collect[0] == "energy":
            del collect[0]        
        for line in collect:
            line = line.split(":")[-1].split(" ")
            gaplist.append(line[1]-line[0])
            
def corrAverage(BC, J, D, L, P, m, phys):
    folder = creatDir(BC, J, D, L, P, m, phys)
    name = creatName(BC, J, D, L, P, m, phys)
    myTarPath = folder[2] + "/" + name[2]

    corrDic = {}
    try:
        with open(myTarPath, "r") as targetFile:
            collect = targetFile.readlines()
            collect = [line.strip() for line in collect]
            if collect and collect[0] == "corr1":
                del collect[0]
            for line in collect:
                line = line.split(":")[-1].split(" ")
                for data in line:
                    if data:
                        parts = data.split(",")
                        if len(parts) >= 3:
                            try:
                                x1, x2 = int(parts[0]), int(parts[1])
                                corr = float(parts[2])
                                key = x2 - x1
                                if key not in corrDic:
                                    corrDic[key] = []
                                corrDic[key].append(corr)
                            except ValueError:
                                continue  # 忽略無法轉換的數據
    except FileNotFoundError:
        print(f"File not found: {myTarPath}")
        return False, False, False
    save_corrDistribute(corrDic, BC, J, D, L, P, m, phys)
    corr = {}
    sample = {}
    error = {}
    for key, values in corrDic.items():
        sample[key] = len(values)
        corr[key] = np.mean(values)
        error[key] = np.std(values, ddof=1) if len(values) > 1 else 0.0

    return corr, sample, error

def save_corrDistribute(corrDic, BC, J, D, L, P, m, phys):
    folder = creatDir(BC, J, D, L, P, m, phys)
    name = creatName(BC, J, D, L, P, m, phys)
    # print( f"folder[4]:{folder[4]}")
    corrDisBase = folder[5] + "/" + name[5]
    for key, values in corrDic.items():
        context = ""
        if BC == "OBC":
            context = f"C_etoe=<S(0)S(dx={key})>\n"
        elif BC == "PBC":
            context = f"C_bulk=<S(0)S(dx={key})>\n"
        corrDisPath = corrDisBase.replace("dx_re", f"dx={key}")
        for value in values:
            context += f"{value}\n"
        
        if context == f"C_etoe=<S(0)S(dx={key})>\n" or context ==  f"C_bulk=<S(0)S(dx={key})>\n":
            continue
        else:
            if not os.path.exists(corrDisPath):
                os.makedirs(os.path.dirname(corrDisPath), exist_ok=True)
            with open(corrDisPath, "w") as targetFile:
                targetFile.write(context)

def save_corr(BC, J, D, L, P, m, phys):
    corr, sample, error = corrAverage(BC, J, D, L, P, m, phys)
    l = int(L.replace("L", ""))
    # print("l",l)
    # print("corr",corr)
    # print(corr[l-1], sample[l-1], error[l-1])
    if corr == False:
        print(f"Error: No data found for {BC}, {J}, {D}, {L}, {P}, {m}, {phys}")
        return
    folder = creatDir(BC, J, D, L, P, m, phys)
    name = creatName(BC, J, D, L, P, m, phys)
    # print( f"folder[4]:{folder[4]}")
    myTarPathBase = folder[4] + "/" + name[4]
    # print(f"myTarPath:{myTarPath}")

    for key in corr.keys():
        if corr[key] == None:
            continue
        myTarPath = myTarPathBase.replace("dx_re", f"dx={key}")
        if BC == "OBC":
            context = f"C_etoe=<S(0)S(dx={key})>,erro,sample\n" + f"{corr[key]},{error[key]},{sample[key]}"
        elif BC == "PBC":
            context = f"C_bulk=<S(0)S(dx={key})>,erro,sample\n" + f"{corr[key]},{error[key]},{sample[key]}"
        if not os.path.exists(myTarPath):
            os.makedirs(os.path.dirname(myTarPath), exist_ok=True)
        with open(myTarPath, "w") as targetFile:
            targetFile.write(context)
        # # print(key, corr[key], sample[key], error[key])
        # print(f"myTarPath:{myTarPath.replace("dx_re", f"dx={key}")}")
        # with open(myTarPath, "w") as targetFile:
        #     targetFile.write("key,corr,sample,error\n")


# def save_ZL():
    
# def save_gap():

def list_txt_files(directory):
    txt_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.txt'):
                full_path = os.path.join(root, file)
                txt_files.append(full_path)
    return txt_files

if __name__ == "__main__":
    file = "/home/aronton/tSDRG_random/Subpy/parameterRead/2025/2025_6_12"
    txt_files = list_txt_files(file)
    # file = "/home/aronton/tSDRG_random/Subpy/parameterRead/2025/2025_6_4/Spin15_L24_Jdis010_Dim000_P10_BC=PBC_chi40_partition=scopion1_seed1=1_seed2=10000_ds=50_task=submit_H12_M12_S6.txt"
    arg = []
    # Jstr = [f"Jdis{str(i).zfill(3)}" for i in range(int(J),int(J)+1)]
    # Jstr = [f"Jdis{str(i).zfill(3)}" for i in range(120,150,30)]

    # Dstr = [f"Dim{str(i).zfill(3)}" for i in range(101)]
    # Lstr = [f"L{num}" for num in range(31, 255, 32)]  # 只有 L512
    # Lstr = [f"L{num}" for num in range(31, 127, 8)]  # 只有 L512
    for file in txt_files:
    # file = "/home/aronton/tSDRG_random/Subpy/parameterRead/2025/2025_5_31/Spin15_L191_Jdis010_Dim000_P10_BC=OBC_chi40_partition=scopion3_seed1=1_seed2=5000_ds=100_task=submit_H21_M43_S55.txt"
        try:
            a = scriptCreator.para("read",file)
        except Exception as e:
            print(f"Error reading file {file}: {e}")
            continue
        parameterlist = a.para    
        para=scriptCreator.paraList1(parameterlist["L"],parameterlist["J"],parameterlist["D"],parameterlist["S"])
        BC = parameterlist["BC"]
        Pdis = parameterlist["Pdis"]
        chi = "m" + str(parameterlist["chi"])
        # s1 = int(parameterlist["S"]["S1"])
        # s2 = int(parameterlist["S"]["S2"])
        # s1 = int(sys.argv[2])
        # s2 = int(sys.argv[3])
        # BC = "OBC"
        # Pdis = 10
        # chi = "m40"
        # Pdis = parameterlist["Pdis"]
        # chi = "m" + str(parameterlist["chi"])
        s1 = int(parameterlist["S"]["S1"])
        s2 = int(parameterlist["S"]["S2"])
        # corr, sample, error = corrAverage(BC, "Jdis010", "Dim000", "L31", "P10", "m40", "corr1")
        # save_corr(BC, J, D, L, f"P{Pdis}", f"{chi}", "corr1")    
        for s in ["corr1"]:
            for L in para.L_str:
                for J in para.J_str:
                    corr, sample, error = corrAverage(BC, J, "Dim000", L, "P10", "m40", s)
                    save_corr(BC, J, "Dim000", L, f"P{Pdis}", f"{chi}", "corr1")    
                    print(corr)
                    print(sample)
                    print(error)
                        # arg.append((BC, J, para.D_str[0], L, f"P{Pdis}", f"{chi}", s, 1, 10000))
    # for s in ["corr1"]:
    #     for L in para.L_str:
    #         for J in para.J_str:
    # arg.append((BC, J, D, L, f"P{Pdis}", f"{chi}", s, 1, 10000))
    # print(arg)     
    # corr, sample, error = corrAverage(BC, "Jdis010", "Dim000", "L31", "P10", "m40", "corr1")
    # save_corr(BC, J, D, L, f"P{Pdis}", f"{chi}", "corr1")    


File not found: /home/aronton/tSDRG_random/tSDRG/Main_15/data_collect_old/OBC/Jdis030/Dim000/L7_P10_m40/corr1_L7_P10_m40_Jdis030_Dim000.txt
File not found: /home/aronton/tSDRG_random/tSDRG/Main_15/data_collect_old/OBC/Jdis030/Dim000/L7_P10_m40/corr1_L7_P10_m40_Jdis030_Dim000.txt
Error: No data found for OBC, Jdis030, Dim000, L7, P10, m40, corr1
False
False
False
File not found: /home/aronton/tSDRG_random/tSDRG/Main_15/data_collect_old/OBC/Jdis080/Dim000/L7_P10_m40/corr1_L7_P10_m40_Jdis080_Dim000.txt
File not found: /home/aronton/tSDRG_random/tSDRG/Main_15/data_collect_old/OBC/Jdis080/Dim000/L7_P10_m40/corr1_L7_P10_m40_Jdis080_Dim000.txt
Error: No data found for OBC, Jdis080, Dim000, L7, P10, m40, corr1
False
False
False
File not found: /home/aronton/tSDRG_random/tSDRG/Main_15/data_collect_old/OBC/Jdis030/Dim000/L15_P10_m40/corr1_L15_P10_m40_Jdis030_Dim000.txt
File not found: /home/aronton/tSDRG_random/tSDRG/Main_15/data_collect_old/OBC/Jdis030/Dim000/L15_P10_m40/corr1_L15_P10_m40_Jdis0

In [1]:
myTarPath = "/home/aronton/tSDRG_random/tSDRG/Main_15/data_collect/OBC/Jdis010/Dim000/L7_P10_m40/corr1_L7_P10_m40_Jdis010_Dim000.txt"
with open(myTarPath, "r") as targetFile:
    collect = targetFile.readlines()
    collect = [line.strip() for line in collect]  # Remove empty lines and strip whitespace
print(collect[1])

1:x1,x2,corr 0,1,-3.138726698282046 0,2,2.091195460284133 0,3,-1.835099502286222 0,4,1.63334964113194 0,5,-1.370862052416925 0,6,1.484102492089434


In [ ]:
-0.14058714223621982 5000 0.7500201012887087